# Indexación y selección de datos

Aquí veremos medios  para acceder y modificar valores en Pandas ``Series`` y ``DataFrame`` objetos. Si ha utilizado los patrones NumPy, los patrones correspondientes en Pandas le resultarán muy familiares, aunque hay algunas peculiaridades que debe tener en cuenta.

Comenzaremos con el caso simple del objeto unidimensional ``Series``, y luego pasar al objeto bidimensional más complicado ``DataFrame``.

## Selección de datos en serie

Un objeto ``Series`` actúa en muchos aspectos como una matriz NumPy unidimensional y, en muchos aspectos, como un diccionario estándar de Python. Si tenemos en cuenta estas dos analogías superpuestas, nos ayudará a comprender los patrones de indexación y selección de datos en estas matrices.

### Serie como diccionario

Como un diccionario, el objecto ``Series`` proporciona un mapeo de una colección de claves a una colección de valores: 

In [1]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

También podemos usar expresiones y métodos de Python similares a un diccionario para examinar las claves/índices y valores:

In [3]:
'a' in data

True

In [4]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario. Así como puede extender un diccionario asignándolo a una nueva clave, puede extender un ``Series`` asignando a un nuevo valor de índice:

In [6]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas toma decisiones sobre el diseño de la memoria y la copia de datos que podría ser necesario; el usuario generalmente no necesita preocuparse por estos problemas.

### Serie como matriz unidimensional


Las ``Series`` se basa en esta interfaz similar a un diccionario y proporciona una selección de elementos de estilo de matriz a través de los mismos mecanismos básicos que las matrices NumPy, es decir,  *slices*, *masking*, and *fancy indexing*. Ejemplos de estos son los siguientes:

In [7]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [9]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [10]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Entre estos, el corte puede ser la fuente de mayor confusión. Tenga en cuenta que al cortar con un índice explícito (es decir, ``data['a':'c']``), el índice final se incluye en el segmento, mientras que al dividir con un índice implícito (es decir, ``data[0:2]``), el índice final se excluye del segmento.

### Indexadores: loc, iloc y ix

Estas convenciones de segmentación e indexación pueden ser una fuente de confusión. Por ejemplo, si su ``Series`` tiene un índice entero explícito, una operación de indexación como ``data[1]`` utilizará los índices explícitos, mientras que una operación de corte como ``data[1:3]`` utilizará el índice implícito de estilo Python. 

In [11]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [12]:
# explicit index when indexing
data[1]

'a'

In [13]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object


Debido a esta posible confusión en el caso de los índices de enteros, Pandas proporciona algunos *indexer* que exponen explícitamente ciertos esquemas de indexación. Estos no son métodos funcionales, sino atributos que exponen una interfaz de corte particular a los datos en el ``Series``.

Primero el atributo ``loc`` permite la indexación y el corte que siempre hace referencia al índice explícito:

In [23]:
data.loc[1]

'a'

In [18]:
data.loc[1:3]

1    a
3    b
dtype: object

El atributo ``iloc`` permite la indexación y el corte que siempre hace referencia al índice implícito de estilo Python:

In [22]:
data.iloc[1]

'b'

In [24]:
data.iloc[1:3]

3    b
5    c
dtype: object



Un tercer atributo de indexación,``ix``, es un híbrido de los dos, y por objetos ``Series`` es equivalente al estándar ``[]`` basada en indexación. El propósito de ``ix`` indexador se hará más evidente en el contexto de objetos ``DataFrame``, que discutiremos en un momento.

Un principio rector del código de Python es que "explícito es mejor que implícito". El carácter explícito de ``loc`` and ``iloc`` hacerlos muy útiles para mantener un código limpio y legible; especialmente en el caso de índices enteros, recomiendo usarlos para hacer que el código sea más fácil de leer y comprender, y para evitar errores sutiles debido a la convención mixta de indexación/corte.

## Selección de datos en DataFrame

Recuerda que un ``DataFrame`` actúa en muchos aspectos como una matriz bidimensional o estructurada y, en otros, como un diccionario de  ``Series`` estructuras que comparten el mismo índice. Estas analogías pueden ser útiles para tener en cuenta a medida que exploramos la selección de datos dentro de esta estructura. 

### DataFrame como diccionario

La primera analogía que consideraremos es la ``DataFrame`` como un diccionario de objetos ``Series`` relacionados. Volvamos a nuestro ejemplo de áreas y poblaciones de estados:

In [31]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pob = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'extension':area, 'poblacion':pob})
data

,extension,poblacion
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


El individuo ``Series`` que forman las columnas de la ``DataFrame`` puede acceder a través de la indexación de estilo de diccionario del nombre de la columna:

In [30]:
data['extension']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: extension, dtype: int64

De manera equivalente, podemos usar el acceso de estilo de atributo con nombres de columna que son cadenas:

In [33]:
data.extension

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: extension, dtype: int64

Este acceso de columna de estilo de atributo en realidad accede exactamente al mismo objeto que el acceso de estilo de diccionario:

In [35]:
data.extension is data['extension']

True

Aunque esta es una abreviatura útil, ¡tenga en cuenta que no funciona para todos los casos! Por ejemplo, si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con los métodos del ``DataFrame``, este acceso de estilo de atributo no es posible. por ejemplo, el ``DataFrame`` tiene un método ``pop()``, así ``data.pop`` apuntará a esto en lugar de a la ``"pop"`` columna:

In [40]:
#data.pop is data['pop']
data

,extension,poblacion,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


En particular, debe evitar la tentación de probar la asignación de columnas a través de atributos (es decir, usar ``data['pop'] = z`` en vez de ``data.pop = z``).

Como con el objeto ``Series`` discutidos anteriormente, esta sintaxis de estilo de diccionario también se puede usar para modificar el objeto, en este caso agregando una nueva columna:

In [39]:
data['density'] = data['poblacion'] / data['extension']
data

,extension,poblacion,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


### DataFrame como matriz bidimensional 

También podemos ver el ``DataFrame`` como una matriz bidimensional mejorada. Podemos examinar la matriz de datos subyacente sin procesar usando el atributo ``values``: 

In [41]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Con esta imagen en mente, se pueden hacer muchas observaciones similares a arreglos familiares en el ``DataFrame`` sí mismo. Por ejemplo, podemos transponer la totalidad ``DataFrame`` para intercambiar filas y columnas:

In [42]:
data.T

,California,Texas,New York,Florida,Illinois
extension,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
poblacion,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Cuando se trata de la indexación de objetos ``DataFrame``, sin embargo, está claro que la indexación de columnas de estilo diccionario impide nuestra capacidad de tratarlo simplemente como una matriz NumPy. En particular, pasar un solo índice a una matriz accede a una fila:

In [43]:
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

y pasando un solo "índice" a un ``DataFrame`` accede a una columna:

In [45]:
data['extension']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: extension, dtype: int64


Por lo tanto, para la indexación de estilo de matriz, necesitamos otra convención. Aquí Pandas vuelve a utilizar el ``loc``, ``iloc``, and ``ix`` indexadores mencionados anteriormente. Utilizando el indexador ``iloc``, podemos indexar la matriz subyacente como si fuera una matriz NumPy simple (usando el índice implícito de estilo Python), pero el ``DataFrame`` las etiquetas de índice y columna se mantienen en el resultado:

In [61]:
# data.iloc[f,c]
data.iloc[:3,:2]

,extension,poblacion
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


Del mismo modo, usando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar a una matriz pero usando el índice explícito y los nombres de las columnas:

In [67]:
data.loc[:'Florida', :'density']

,extension,poblacion,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


El indexer ``ix`` permite un híbrido de estos dos enfoques:

In [74]:
#data.ix[:3, :'poblacion']

Cualquiera de los patrones familiares de acceso a datos de estilo NumPy se puede utilizar dentro de estos indexadores. por ejemplo, en el indexador ``loc`` podemos combinar el enmascaramiento y la indexación elegante como se muestra a continuación:

In [75]:
data.loc[data.density > 100, ['poblacion', 'density']]

,poblacion,density
New York,19651127,139.076746
Florida,19552860,114.806121


Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; esto se hace de la manera estándar a la que podría estar acostumbrado al trabajar con NumPy:

In [76]:
data.iloc[0, 2] = 90
data

,extension,poblacion,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Para desarrollar su fluidez en la manipulación de datos de Pandas, le sugiero pasar algún tiempo con un simple ``DataFrame`` y explorar los tipos de indexación, corte, enmascaramiento e indexación elegante que permiten estos diversos enfoques de indexación.

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contrarias a la discusión anterior, pero que sin embargo pueden ser muy útiles en la práctica. Primero, mientras que *indexing* se refiere a las columnas, el *slicing* se refiere a las filas: 

In [77]:
data['Florida':'Illinois']

,extension,poblacion,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Dichos segmentos también pueden referirse a filas por número en lugar de por índice:

In [80]:
data[3:5]

,extension,poblacion,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


De manera similar, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [81]:
data[data.density > 100]

,extension,poblacion,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


Estas dos convenciones son sintácticamente similares a las de una matriz NumPy, y aunque es posible que no encajen con precisión en el molde de las convenciones de Pandas, son bastante útiles en la práctica.